In [2]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

import json
with open('intents.json') as file:
    data = json.load(file)

words = []
labels = []
docs_x = [] #patterns 
docs_y = [] #intents

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(pattern)
        docs_y.append(intent['tag'])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
    
words = [stemmer.stem(w.lower()) for w in words]
words = sorted(list(set(words)))

labels = sorted(labels)

#Training


In [2]:
!pwd

/Users/laxmanjeergal/Desktop/ml


In [12]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/laxmanjeergal/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True